In [7]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
from utilities import statistics, post_processing

In [2]:
tstat_path =  '/home/yolanda/workspace/tcp_temp_complete_tagged_ok'

sc = pyspark.SparkContext()
sqlCxt = SQLContext(sc)
df = sqlCxt.read.format('com.databricks.spark.csv').options(header='true', delimiter=" ", inferschema='true').load(tstat_path)

In [9]:
df_connect_pkts = statistics.get_total_pkts_flowid(df)
df_ranked_pkts = statistics.compute_ranking(df_connect_pkts, None, 4)

AnalysisException: 'Cannot resolve column name "s_pkts_all:17" among (#c_ip:1, c_port:2, s_ip:15, s_port:16, tag:132, sum(s_pkts_all:17), sum(c_pkts_all:3), total_pkts);'

In [21]:
df.printSchema()

root
 |-- #c_ip:1: string (nullable = true)
 |-- c_port:2: integer (nullable = true)
 |-- c_pkts_all:3: integer (nullable = true)
 |-- c_rst_cnt:4: integer (nullable = true)
 |-- c_ack_cnt:5: integer (nullable = true)
 |-- c_ack_cnt_p:6: integer (nullable = true)
 |-- c_bytes_uniq:7: integer (nullable = true)
 |-- c_pkts_data:8: integer (nullable = true)
 |-- c_bytes_all:9: integer (nullable = true)
 |-- c_pkts_retx:10: integer (nullable = true)
 |-- c_bytes_retx:11: integer (nullable = true)
 |-- c_pkts_ooo:12: integer (nullable = true)
 |-- c_syn_cnt:13: integer (nullable = true)
 |-- c_fin_cnt:14: integer (nullable = true)
 |-- s_ip:15: string (nullable = true)
 |-- s_port:16: integer (nullable = true)
 |-- s_pkts_all:17: integer (nullable = true)
 |-- s_rst_cnt:18: integer (nullable = true)
 |-- s_ack_cnt:19: integer (nullable = true)
 |-- s_ack_cnt_p:20: integer (nullable = true)
 |-- s_bytes_uniq:21: integer (nullable = true)
 |-- s_pkts_data:22: integer (nullable = true)
 |-- s_